In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.layers.recurrent import SimpleRNN
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

Using TensorFlow backend.


In [0]:
df1 = pd.read_csv('train_genba.tsv', sep='\t')
df2 = pd.read_csv('train_goto.tsv', sep='\t')
df = pd.merge(df1, df2, on='pj_no', how='outer')

del df1
del df2

dftest1 = pd.read_csv('test_genba.tsv', sep='\t')
dftest2 = pd.read_csv('test_goto.tsv', sep='\t')
dftest = pd.merge(dftest1, dftest2, on='pj_no', how='outer')

del dftest1
del dftest2

df = df.reset_index(drop=False)
dftest = dftest.reset_index(drop=False)

# 出力用
dftest_id = dftest['id']

#欠損値
df = df.fillna(value=(-1))
df = df.replace('Nan', (-1))
dftest = dftest.fillna(value=(-1))
dftest = dftest.replace('Nan', (-1))

factorize = []

# string => float, int
for i, j in zip(df.columns, df.dtypes):
    if j == 'object':
        factorize.append(i)
    
for i in factorize:
    id_1, id_2 = df[i].factorize()
    id_3 = pd.DataFrame({i: id_1})
    df[i] = id_3
    dftest[i] = id_3

# X, Y
dfall = df.drop('keiyaku_pr', axis=1)
X = dfall.as_matrix()
Y = df['keiyaku_pr'].as_matrix()
XX = dftest.as_matrix()

scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)
XX = scaler.fit_transform(XX)

maxlen = 158

X = np.array(X).reshape(len(X), maxlen, 1)
#XX = np.array(X).reshape(len(XX), maxlen, 1)

NameError: ignored

In [0]:
# モデルの構築
n_in = 1
n_hidden = 30
n_out = 1


def weight_variable(shape, name=None):
    return np.random.normal(scale=.01, size=shape)


early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

model = Sequential()
model.add(LSTM(n_hidden,
               kernel_initializer=weight_variable,
               input_shape=(maxlen, n_in)))
model.add(Dense(n_out, kernel_initializer=weight_variable))
model.add(Activation('linear'))

optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
model.compile(loss='mean_squared_error',
              optimizer=optimizer)

In [0]:
epochs = 10
batch_size = 10

model.fit(X, Y,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=[early_stopping])

W0622 06:36:29.036080 140156273194880 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0622 06:36:29.722588 140156273194880 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0622 06:36:29.810497 140156273194880 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.

W0622 06:36:29.883471 140156273194880 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:2741: The name tf.Session

Epoch 1/10
6461/6461 [==============================] - 55s 9ms/step - loss: 821008552124681.0000
Epoch 2/10
  20/6461 [..............................] - ETA: 50s - loss: 785170789564416.0000

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


6461/6461 [==============================] - 58s 9ms/step - loss: 821007380638241.0000
Epoch 3/10
6461/6461 [==============================] - 57s 9ms/step - loss: 821006254775649.2500
Epoch 4/10
6461/6461 [==============================] - 59s 9ms/step - loss: 821005140899380.1250
Epoch 5/10
6461/6461 [==============================] - 59s 9ms/step - loss: 821004024145978.0000
Epoch 6/10
6461/6461 [==============================] - 59s 9ms/step - loss: 821002908880479.5000
Epoch 7/10
6461/6461 [==============================] - 58s 9ms/step - loss: 821001801415438.6250
Epoch 8/10
6461/6461 [==============================] - 57s 9ms/step - loss: 821000683877836.0000
Epoch 9/10
6461/6461 [==============================] - 57s 9ms/step - loss: 820999573460358.2500
Epoch 10/10
6461/6461 [==============================] - 58s 9ms/step - loss: 820998464133490.5000


In [0]:
XX = np.array(XX).reshape(len(XX), maxlen, 1)

YY = model.predict(XX) * 1000000
df_pre = pd.DataFrame(YY)
df_pre.head()

,0
0,2.033566e+08
1,2.033566e+08
2,2.033566e+08
3,2.033566e+08
4,2.033566e+08


In [0]:
df_pre = df_pre.astype(int)
df_pre = df_pre.round()
df_pre = pd.concat(objs=[dftest_id, df_pre], axis=1)
df_pre.head()

,id,0
0,test_0000,203356552
1,test_0001,203356552
2,test_0002,203356552
3,test_0003,203356552
4,test_0004,203356552


In [0]:
df_pre.to_csv('test_pre.tsv', sep='\t', index=False, header=False)